In [770]:
import requests
import pandas as pd
import numpy as np
from coinbase.rest import RESTClient
import datetime
import json
import time
import os
import plotly.express as px



In [780]:
fdump = 'test.csv'
import csv
file =  open(fdump, mode="w", newline="")
writer = csv.writer(file)
writer.writerow(["time", "cb bid", "cb ask", "kk bid", "kk ask"])

34

In [772]:

prod = "MATIC-USD"

api_key = os.environ.get("COINBASE_API_TOKEN")
api_secret = os.environ.get("COINBASE_PRIVATE_TOKEN")

client = RESTClient(api_key=api_key, api_secret=api_secret)

prodkk=''.join(char for char in prod if char.isalnum())
url = "https://api.kraken.com/0/public/Depth?pair={}".format(prodkk)

payload = {}
headers = {
  'Accept': 'application/json'
}



In [773]:
response = requests.request("GET", url, headers=headers, data=payload)
ts = time.time()
cb_pb = client.get_public_product_book(prod, 100)['pricebook']

In [774]:
kk_m = json.loads(response.text)
cb_bids = cb_pb['bids']
cb_asks = cb_pb['asks']

for ii in range(len(cb_bids)):
    cb_bids[ii] = cb_bids[ii].__dict__
for ii in range(len(cb_asks)):
    cb_asks[ii] = cb_asks[ii].__dict__

In [775]:

cb_bids = pd.DataFrame.from_dict(cb_bids)
cb_asks = pd.DataFrame.from_dict(cb_asks)

cb_bids['price'] = cb_bids['price'].astype(float)
cb_bids['size'] = cb_bids['size'].astype(float)

cb_asks['price'] = cb_asks['price'].astype(float)
cb_asks['size'] = cb_asks['size'].astype(float)

In [776]:
kk_bids = pd.DataFrame(kk_m['result'][prodkk]['bids'])
kk_bids = kk_bids.drop(2, axis=1)
kk_bids.columns=['price', 'size']
kk_bids['price'] = kk_bids['price'].astype(float)
kk_bids['size'] = kk_bids['size'].astype(float)

kk_asks = pd.DataFrame(kk_m['result'][prodkk]['asks'])
kk_asks = kk_asks.drop(2, axis=1)
kk_asks.columns=['price', 'size']
kk_asks['price'] = kk_asks['price'].astype(float)
kk_asks['size'] = kk_asks['size'].astype(float)

In [777]:
from scipy.stats  import hmean
def weighted_sum(df):
    return hmean(df['price'], weights=df['size'])


In [778]:
print("cb bids\t", weighted_sum(cb_bids))
print("cb asks\t", weighted_sum(cb_asks))


print()
print("kk bids\t", weighted_sum(kk_bids))
print("kk asks\t", weighted_sum(kk_asks))


print()
print("cb\t", (weighted_sum(cb_bids) + weighted_sum(cb_asks))/2)
print("kk\t", (weighted_sum(kk_bids) + weighted_sum(kk_asks))/2)

cb bids	 0.46532353737249704
cb asks	 0.4995531703291106

kk bids	 0.45829896831647693
kk asks	 0.48897141688890583

cb	 0.4824383538508038
kk	 0.4736351926026914


In [779]:

writer.writerow([
    np.max(cb_bids['price']),
    np.min(cb_asks['price']),
    ts,
    np.max(kk_bids['price']),
    np.min(kk_asks['price']),
])


47